# Learning pyswip

In [1]:
import sys; sys.version
from pyswip import Prolog
import re

sys.path.append("../python")
from POS_tagger import Tagger

pl = Prolog()

PROLEXA_PATH = "../prolog/"

## Imports (consults)

In [2]:
pl.consult(PROLEXA_PATH + "prolexa.pl")

In [38]:
from pyswip import Variable

def handle_utterance_str(text) :
    if text[0] != "'" and text[0] != '"' :
        text = f'"{text}"'
    
    text = text.replace('"', '\"')
    text = text.replace("'", '\"')
    
    return "handle_utterance(1,{},Output)".format(text)

def tag(text) :
    tagged_sent, sent, tags = Tagger().tag(text)
    print(tags)
    return tags

def update_rules(text):
    tags = tag(text)
    text = text.split(' ')
    start = '--> ['
    end = ']'
    f = open(PROLEXA_PATH + "prolexa_grammar.pl", "r")
    lines=f.readlines()
    f.close()
    myiter = iter(lines)
    for idx, line in enumerate(myiter):
        word = text[0]
        if 'DET' in tags and re.match(r"determiner\([a-z],X=>B,X=>H,\[\(H:-B\)\]\)(.*)", line):
            exists = False
            input_word = text[tags.index('DET')]
            for det_idx, det_line in enumerate(lines[idx:]):                                                
                if not(re.match(r"determiner\([a-z],X=>B,X=>H,\[\(H:-B\)\]\)(.*)", det_line)):
                    det_idx = det_idx + idx                
                    break
                line_word = (det_line.split(start))[1].split(end)[0]  
                if input_word == line_word:                    
                    exists = True
                    break 

            if not exists:
                new_line = "determiner(s,X=>B,X=>H,[(H:-B)]) --> [{}].\n".format(input_word) 
                lines.insert(det_idx, new_line)
                next(myiter, None)
                if text:
                    text.pop(0)
                if tags:
                    tags.pop(0)
                    
#         if 'NOUN' in tags and re.match(r"pred\((.*)[1],\[(.*)\]\)\.)", line):
#             exists = False
#             input_word = text[tags.index('DET')]
#             for noun_idx, noun_line in enumerate(lines[idx:]):                                                
#                 if not(re.match(r"pred\((.*)[1],\[(.*)\]\)\.)", det_line)):
#                     noun_idx = noun_idx + idx                
#                     break
#                 line_word = (noun_line.split(start))[1].split(end)[0]  
#                 if input_word == line_word:                    
#                     exists = True
#                     break 

#             if not exists:
#                 new_line = "determiner(s,X=>B,X=>H,[(H:-B)]) --> [{}].\n".format(input_word) 
#                 lines.insert(noun_idx_idx, new_line)
#                 next(myiter, None)
#                 if text:
#                     text.pop(0)
#                 if tags:
#                     tags.pop(0)


    f = open(PROLEXA_PATH + "prolexa_grammar.pl", "w")
    lines = "".join(lines)
    f.write(lines)
    f.close()
            

def escape_and_call_prolexa(text) :
    libPrefix = "prolexa:"
    update_rules(text)
    return pl.query(libPrefix + handle_utterance_str(text))


out = escape_and_call_prolexa('the human is not mortal')
list(out)

2020-05-25 10:22:15,595 loading file C:\Users\dwhet\.flair\models\en-pos-ontonotes-v0.4.pt
['DET', 'NOUN', 'VERB', 'ADV', 'ADJ']


[{'Output': 'I heard you say,  the human is not mortal , could you rephrase that please?'}]

In [ ]:

#Tagger().tag("I have no idea what I'm doing")
#Tagger().tag("Penguins re mrotal")
Tagger().tag("every human is mortal")

In [ ]:
from pyswip import Prolog, Functor, Variable, Query, call

pl = Prolog()
pl.consult("../prolog/prolexa.pl")

In [ ]:
handle_utterance = Functor("handle_utterance", 3)
Output = Variable()
q = Query(handle_utterance(1, "Peter is mortal.", Output), )

while q.nextSolution() :
    print(Output.value)

In [ ]:
q.nextSolution()

In [ ]:
q.nextSolution()

In [ ]:
pl.consult("trains.pl")
STATIONS = [ans["S"] for ans in pl.query("station(S)")]
STATIONS

In [ ]:
pl.assertz("father(michael,john)")
pl.assertz("father(michael,gina)")
pl = Prolog()

for soln in pl.query("father(X,Y)"):
    print(soln["X"], "is the father of", soln["Y"])

# michael is the father of john
# michael is the father of gina

childrenOfMichael = pl.query("father(michael,X)")
list(childrenOfMichael)